In [19]:
import hail as hl
hl.init()

Loading BokehJS ...

24/06/10 17:51:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.4
SparkUI available at http://192.168.0.16:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.130-bea04d9c79b5
LOGGING: writing to /Users/benoppenheimer/Dropbox (Partners HealthCare)/ancestry_analysis_2/code/hail-20240610-1751-0.2.130-bea04d9c79b5.log


In [1]:
import os
import numpy as np
import pandas as pd

# Set-up Functions and Variables

In [2]:
all_sample_list = set(np.loadtxt("../ashkenazi_data/ref_samples.txt", dtype=str))

In [3]:
reich_samples = []
with open("../ashkenazi_data/EuropeFullyPublic/original_eigenstrat/vdata.ind") as f:
    for line in f.readlines():
        sample = line.strip().split()
        reich_samples.append(sample)
reich_df = pd.DataFrame(data=reich_samples, columns=["Sample_Name", "Gender", "Ancestry"])
reich_df.head()

,Sample_Name,Gender,Ancestry
0,SA1004,F,Khomani
1,SA063,F,Khomani
2,SA010,F,Khomani
3,SA064,M,Khomani
4,SA073,F,Khomani


In [4]:
reich_samples = []
with open("../ashkenazi_data/EuropeFullyPublic/convertf_plink/vdata.fam") as f:
    for line in f.readlines():
        sample = line.strip().split()
        reich_samples.append(sample)
orginal_fam = pd.DataFrame(data=reich_samples)
orginal_fam.head()

,0,1,2,3,4,5
0,1,SA1004,0,0,2,1
1,2,SA063,0,0,2,1
2,3,SA010,0,0,2,1
3,4,SA064,0,0,1,1
4,5,SA073,0,0,2,1


In [5]:
orginal_fam.to_csv("../ashkenazi_data/EuropeFullyPublic/convertf_plink/vdata.fam", sep=" ", header=False, index=False)

In [6]:
reich_samples_to_include = reich_df[reich_df.Ancestry.str.startswith("Ashkenazi")]
reich_samples_to_include.loc[:, "plink_name"] = (reich_samples_to_include.index + 1).astype(str) + "\t" + \
                                                reich_samples_to_include.Sample_Name
reich_samples_to_include

/var/folders/w0/3_6gnh594j99k2l1f70tp2tm0000gn/T/ipykernel_3958/4225007770.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reich_samples_to_include.loc[:, "plink_name"] = (reich_samples_to_include.index + 1).astype(str) + "\t" + \


,Sample_Name,Gender,Ancestry,plink_name
1633,AshkenaziJew5779,F,Ashkenazi_Jew,1634\tAshkenaziJew5779
1653,AshkenaziJew5782,M,Ashkenazi_Jew,1654\tAshkenaziJew5782
1797,AshkenaziJew5783,M,Ashkenazi_Jew,1798\tAshkenaziJew5783
1800,AshkenaziJew5704,F,Ashkenazi_Jew,1801\tAshkenaziJew5704
1807,AshkenaziJew5728,F,Ashkenazi_Jew,1808\tAshkenaziJew5728
1813,AshkenaziJew5790,M,Ashkenazi_Jew,1814\tAshkenaziJew5790
1905,AshkenaziJew5788,M,Ashkenazi_Jew,1906\tAshkenaziJew5788


In [7]:
reich_samples_to_include.shape[0]

7

In [8]:
reich_samples_to_include.Sample_Name.to_csv("../ashkenazi_data/EuropeFullyPublic/samples_to_include.txt",
                                            header=False, index=False)

In [9]:
jew_samples = pd.read_excel("../ashkenazi_data/jew/Jew_paper_samples.xlsx")
jew_samples.head()

/opt/homebrew/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Sample ID,Population,Country / collected in,Continent,chip
0,EthiopiaAm50,Ethiopians,Ethiopia,Africa,660k
1,EthiopiaAm52,Ethiopians,Ethiopia,Africa,660k
2,EthiopiaAm58,Ethiopians,Ethiopia,Africa,660k
3,EthiopiaAm59,Ethiopians,Ethiopia,Africa,660k
4,EthiopiaAm76,Ethiopians,Ethiopia,Africa,660k


In [10]:
jew_samples_to_include = jew_samples[jew_samples.Population.str.startswith("Ashkenazy")]
jew_samples_to_include.loc[:, "plink_name"] = jew_samples_to_include["Sample ID"] + " " + \
                                              jew_samples_to_include["Sample ID"]
jew_samples_to_include.head()

/var/folders/w0/3_6gnh594j99k2l1f70tp2tm0000gn/T/ipykernel_3958/2019007429.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jew_samples_to_include.loc[:, "plink_name"] = jew_samples_to_include["Sample ID"] + " " + \


,Sample ID,Population,Country / collected in,Continent,chip,plink_name
180,ashkenazy10e,Ashkenazy_Jews,Russia,Europe,610k,ashkenazy10e ashkenazy10e
181,ashkenazy10w,Ashkenazy_Jews,Netherlands,Europe,610k,ashkenazy10w ashkenazy10w
182,ashkenazy1e,Ashkenazy_Jews,Belorussia,Europe,610k,ashkenazy1e ashkenazy1e
183,ashkenazy1w,Ashkenazy_Jews,Austria,Europe,610k,ashkenazy1w ashkenazy1w
184,ashkenazy2e,Ashkenazy_Jews,Belorussia,Europe,610k,ashkenazy2e ashkenazy2e


In [11]:
jew_samples_to_include.shape[0]

21

In [12]:
jew_samples_to_include["Sample ID"].isin(reich_samples_to_include.Sample_Name).any()

False

In [13]:
jew_samples_to_include.plink_name.to_csv("../ashkenazi_data/jew/samples_to_include.txt", header=False, index=False)

In [14]:
def get_samples_in_cohort(fam_file):
    samples = dict()
    with open(fam_file) as f:
        for line in f.readlines():
            split_line = line.split()
            fam_id = split_line[0]
            sample = split_line[1]
            samples[sample] = fam_id
    return samples

def liftover_vcf_file(cohort):
    input_vcf = cohort.get_suffix_bfile("filtered") + ".vcf"
    mt = hl.import_vcf(input_vcf)
    mt = mt.annotate_rows(new_locus=hl.liftover(mt.locus, 'GRCh38', include_strand=True), old_locus=mt.locus)

    #save not_lifted vcf info
    not_lifted_mt = mt.filter_rows(~hl.is_defined(mt.new_locus) | mt.new_locus.is_negative_strand)
    not_lifted_vcf = cohort.get_suffix_bfile("not_lifted") + ".vcf"
    hl.export_vcf(not_lifted_mt, not_lifted_vcf)

    lifted_mt = mt.filter_rows(hl.is_defined(mt.new_locus) & ~mt.new_locus.is_negative_strand)
    lifted_mt = lifted_mt.key_rows_by(locus=lifted_mt.new_locus.result, alleles=lifted_mt.alleles)
    lifted_mt_filename = cohort.get_suffix_bfile("lifted_mt") + ".vcf"
    hl.export_vcf(lifted_mt, lifted_mt_filename)


class Cohort:

    def __init__(self, original_bfile, is_bfile=True):

        self.original_bfile = original_bfile
        self.original_bfile_name = os.path.basename(self.original_bfile)

        split_dir = self.original_bfile.split("/")
        self.dir = "/".join(split_dir[0:-2])


    def get_suffix_bfile(self, suffix):
        corrected_dir = self.dir + "/" + suffix

        if not os.path.exists(corrected_dir):
            os.mkdir(corrected_dir)

        corrected_bfile = corrected_dir + "/" + self.original_bfile_name + "_" + suffix
        return corrected_bfile

In [15]:
original_bfiles = ["../ashkenazi_data/EuropeFullyPublic/convertf_plink/vdata",
                   "../ashkenazi_data/jew/original_plink/jew_paper_data_dbSNP-b131_pos-b37_1KG_strand"]
cohorts = [Cohort(original_bfile) for original_bfile in original_bfiles]

In [16]:
cohort.original_bfile

NameError: name 'cohort' is not defined

In [20]:
hg19_fasta = "../../hg38_reference_data/Homo_sapiens_assembly19.fasta"

def filter_samples(mt, cohort):

    samples_to_include_file = f"{cohort.dir}/samples_to_include.txt"
    samples_to_include = set(pd.read_csv(samples_to_include_file).iloc[:, 0].to_list())

    set_to_keep = hl.literal(samples_to_include)

    mt = mt.filter_cols(set_to_keep.contains(mt['s']))
    return mt

for i, cohort in enumerate(cohorts):
    print(i)
    mt = hl.import_plink(bed=f"{cohort.original_bfile}.bed",
                     bim=f"{cohort.original_bfile}.bim",
                     fam=f"{cohort.original_bfile}.fam",
                     reference_genome='GRCh37')
    print("original count:", mt.count())
    mt = filter_samples(mt, cohort)
    print("count for Ashkenazi data", mt.count())
    mt.write(f"{cohort.dir}/filtered_mt.mt", overwrite=True)
    print()

0


2024-06-10 17:51:22.459 Hail: INFO: Found 1963 samples in fam file.
2024-06-10 17:51:22.459 Hail: INFO: Found 600841 variants in bim file.
2024-06-10 17:51:23.446 Hail: INFO: Found 1963 samples in fam file.
2024-06-10 17:51:23.446 Hail: INFO: Found 600841 variants in bim file.


original count: (600841, 1963)


2024-06-10 17:51:30.405 Hail: INFO: Found 1963 samples in fam file.
2024-06-10 17:51:30.406 Hail: INFO: Found 600841 variants in bim file.


count for Ashkenazi data (600841, 6)


...


1


2024-06-10 17:52:06.066 Hail: INFO: Found 466 samples in fam file.
2024-06-10 17:52:06.067 Hail: INFO: Found 555737 variants in bim file.
2024-06-10 17:52:08.388 Hail: INFO: Found 466 samples in fam file.
2024-06-10 17:52:08.388 Hail: INFO: Found 555737 variants in bim file.


original count: (555737, 466)



Exception: java.lang.OutOfMemoryError thrown from the UncaughtExceptionHandler in thread "RemoteBlock-temp-file-clean-thread"
Exception in thread "idle-timeout-task" Exception in thread "refresh progress" Exception in thread "dispatcher-event-loop-1" java.lang.OutOfMemoryError: Java heap space
java.lang.OutOfMemoryError: Java heap space
java.lang.OutOfMemoryError: Java heap space
Exception in thread "Spark Context Cleaner" java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.ContextCleaner$$Lambda$937/0x00000008008ad040.get$Lambda(Unknown Source)
	at java.base/java.lang.invoke.DirectMethodHandle$Holder.invokeStatic(DirectMethodHandle$Holder)
	at java.base/java.lang.invoke.LambdaForm$MH/0x0000000800061840.linkToTargetMethod(LambdaForm$MH)
	at org.apache.spark.ContextCleaner.$anonfun$keepCleaning$1(ContextCleaner.scala:195)
	at org.apache.spark.ContextCleaner$$Lambda$778/0x0000000800810440.apply$mcV$sp(Unknown Source)
	at org.apache.spark.util.Utils$.tryOrStopSparkContext(Ut

FatalError: OutOfMemoryError: Java heap space

Java stack trace:
java.lang.OutOfMemoryError: Java heap space
	at 



Hail version: 0.2.130-bea04d9c79b5
Error summary: OutOfMemoryError: Java heap space

# Liftover Bed Files

In [73]:
mt = hl.import_plink(bed=f"{cohort.original_bfile}.bed",
                     bim=f"{cohort.original_bfile}.bim",
                     fam=f"{cohort.original_bfile}.fam",
                     reference_genome='GRCh37')
mt.count()

2024-06-10 14:46:58.933 Hail: INFO: Found 466 samples in fam file.
2024-06-10 14:46:58.934 Hail: INFO: Found 555737 variants in bim file.
2024-06-10 14:46:59.696 Hail: INFO: Found 466 samples in fam file.
2024-06-10 14:46:59.696 Hail: INFO: Found 555737 variants in bim file.


(555737, 466)

In [ ]:
rg37 = hl.get_reference('GRCh37')
rg38 = hl.get_reference('GRCh38')
rg37.add_liftover("grch37_to_grch38.over.chain.gz", rg38)

In [ ]:
for cohort in cohorts:
    liftover_vcf_file(cohort)